In [1]:
%%time

from pandas import DataFrame, Grouper
import pandas as pd
import numpy as np
from datetime import datetime
import matplotlib.pyplot as plt
import sys
from collections import defaultdict
from IPython.display import HTML

CPU times: user 420 ms, sys: 895 ms, total: 1.32 s
Wall time: 269 ms


In [2]:
%%time
# import our own stuff
sys.path.append("../python/src")

from tools import read_water_data, extract_features
from tools import get_oecd_iso3_code_mapping

from waterData import default_features

from worldbankApi import get_regionnames, fetch_series, default_series, get_all_countries

CPU times: user 75.1 ms, sys: 1.07 ms, total: 76.1 ms
Wall time: 75.8 ms


In [3]:
%%time
# set some options

datadir = "../data/"
cachedir = "../data/cache" 

CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 3.1 µs


In [4]:
taz_correspondents_countries = tcc = {
    'Jordan': 'JOR',
    'Egypt, Arab Rep.': 'EGY',
    'Uganda': 'UGA',
    'Rwanda': 'RWA',
    'Kenya': 'KEN',
    'Sudan': 'SDN',
    'India': 'IND',
    'Bangladesh': 'BGD',
    'Colombia': 'COL',
    'Bolivia': 'BOL'
}

inverse_taz_correspondents_countries = itcc = {v: k for k, v in taz_correspondents_countries.items()}


In [5]:
%%time

display("warming up cache")
null = read_water_data(setname="fullset",datadir=datadir,cachedir=cachedir)
del null

'warming up cache'

Reading Datafrom cached file: ../data/cache/fullset.p
CPU times: user 5.34 s, sys: 1.69 s, total: 7.03 s
Wall time: 7.03 s


In [6]:
%%time

# read some data and get some mapping that may needs calculations

# the codemapping is generate over the full dataset to get all oecd-country-codes
# and from a fetch of the countries known to worldbank
codemapping = get_oecd_iso3_code_mapping(cachedir=cachedir,datadir=datadir)

# the data from oecd must be downloaded and saved as zips in the data-dir, check readme for links
oecddf = read_water_data(setname="fullset",datadir=datadir,cachedir=cachedir)


display("all data")
# tell me what it is
oecddf.describe()

Reading countrycode-mapping cached file: ../data/cache/oecdiso3.p
Reading Datafrom cached file: ../data/cache/fullset.p


'all data'

CPU times: user 10.7 s, sys: 2.69 s, total: 13.4 s
Wall time: 13.4 s


,USD_Commitment,USD_Disbursement,USD_Received,USD_Commitment_Defl,USD_Disbursement_Defl,USD_Received_Defl,USD_Adjustment,USD_Adjustment_Defl,USD_AmountUntied,USD_AmountPartialTied,...,TypeRepayment,NumberRepayment,Interest2,USD_Interest,USD_Outstanding,USD_Arrears_Principal,USD_Arrears_Interest,CapitalExpend,PSIflag,PSIAddType
count,3.275182e+06,3.762420e+06,2.075152e+06,3.275072e+06,3.762156e+06,2.075140e+06,60032.000000,60032.000000,1.899677e+06,1.006533e+06,...,549171.000000,564867.000000,333850.000000,1.110691e+06,881558.000000,646025.000000,651855.000000,32902.000000,3301.000000,1218.000000
mean,1.700388e+00,9.906404e-01,4.688217e-01,2.046867e+00,1.047619e+00,5.014058e-01,-0.748338,-0.839550,9.199535e-01,2.892181e-01,...,1.472700,1.854219,394.025889,1.291592e-01,6.420891,0.147550,0.218216,0.000088,1.987882,2.369458
std,2.020509e+01,1.468442e+01,7.989517e+00,2.368979e+01,1.543262e+01,8.453970e+00,11.821947,11.727234,1.252183e+01,4.405014e+00,...,1.179965,0.573195,588.434494,2.429448e+00,55.744203,2.134346,2.654434,0.001916,0.109427,0.919855
min,-1.421530e+03,-2.258730e+02,-5.030000e-01,-1.300270e+03,-2.461000e+02,-6.852520e-01,-1995.000000,-1870.620000,-1.421530e+03,-1.787180e+00,...,0.000000,0.000000,-820.000000,-3.491230e+01,-0.507119,-3.033060,-0.149653,0.000000,1.000000,1.000000
25%,0.000000e+00,3.647640e-03,0.000000e+00,0.000000e+00,3.775977e-03,0.000000e+00,-0.080000,-0.083995,0.000000e+00,0.000000e+00,...,1.000000,2.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000,2.000000,1.000000
50%,2.121690e-02,4.067340e-02,0.000000e+00,2.279000e-02,4.295905e-02,-0.000000e+00,-0.009426,-0.009881,4.364590e-03,0.000000e+00,...,1.000000,2.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000,2.000000,3.000000
75%,1.973465e-01,2.353720e-01,-0.000000e+00,2.188643e-01,2.491832e-01,-0.000000e+00,-0.000870,-0.000894,1.250000e-01,0.000000e+00,...,1.000000,2.000000,750.000000,2.581010e-02,0.848039,0.000000,0.000000,0.000000,2.000000,3.000000
max,6.585080e+03,7.188450e+03,3.690330e+03,7.209450e+03,7.209450e+03,3.637680e+03,0.004700,0.004720,4.087360e+03,8.751230e+02,...,5.000000,52.000000,20000.000000,1.460130e+03,24053.000000,276.494000,900.342000,0.100000,2.000000,3.000000


In [7]:
%%time

# filter out useless / unrelevant data
df = oecddf[oecddf['USD_Commitment'] != 0.0]

df = df[df['USD_Commitment'].notnull()]

# tell me what is left
display("filtered for entries with commitment")
#display(df.describe())

df = df[df['FlowCode'].isin(['13','11'])]

display("only FolowCode 11 and 13")

#display(df.describe())

df = df[df['RecipientCode'].isin(codemapping)]
display("filter regions - only countries that can have an IncomeGroup-Classification")

#display(df.describe())


df = df[df['CommitmentDate'] > datetime(year=1995,month=12,day=31)]
df = df[df['CommitmentDate'] < datetime(year=2020,month=1,day=1)]

display("oecd historyical incomegroupdata is only available for the the years 1996-2019. filtering everything else")

display(df.describe())

# reduce to relevant features
df = extract_features(df,features=default_features)

# free the memory
del oecddf


'filtered for entries with commitment'

'only FolowCode 11 and 13'

'filter regions - only countries that can have an IncomeGroup-Classification'

'oecd historyical incomegroupdata is only available for the the years 1996-2019. filtering everything else'

,USD_Commitment,USD_Disbursement,USD_Received,USD_Commitment_Defl,USD_Disbursement_Defl,USD_Received_Defl,USD_Adjustment,USD_Adjustment_Defl,USD_AmountUntied,USD_AmountPartialTied,...,TypeRepayment,NumberRepayment,Interest2,USD_Interest,USD_Outstanding,USD_Arrears_Principal,USD_Arrears_Interest,CapitalExpend,PSIflag,PSIAddType
count,1.191376e+06,985727.000000,415542.000000,1.191376e+06,985727.000000,415542.000000,2.000000,2.000000,670514.000000,247971.000000,...,26190.000000,26569.000000,34839.000000,206380.000000,199369.000000,167066.000000,171924.000000,3228.000000,177.000000,5.000000
mean,1.601892e+00,0.665446,0.008661,1.733340e+00,0.702637,0.009077,-2.767986,-2.767856,1.436516,0.276916,...,1.563154,1.827430,248.043074,0.015690,0.663979,0.003587,0.153480,0.000220,1.988701,2.600000
std,1.640278e+01,11.582911,0.590555,1.722791e+01,12.191328,0.602097,3.908609,3.908793,16.429945,4.036803,...,1.405181,0.869447,534.832681,0.787091,56.046847,0.146124,1.704386,0.001575,0.105996,0.894427
min,-2.005710e+01,-14.427400,0.000000,-2.180880e+01,-15.655900,0.000000,-5.531790,-5.531790,-20.057100,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000
25%,1.343707e-02,0.007851,0.000000,1.448440e-02,0.008154,0.000000,-4.149888,-4.149823,0.003328,0.000000,...,1.000000,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.000000,3.000000
50%,6.402500e-02,0.038416,0.000000,7.081320e-02,0.040753,0.000000,-2.767986,-2.767856,0.046118,0.000000,...,1.000000,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.000000,3.000000
75%,3.291900e-01,0.169090,-0.000000,3.689710e-01,0.181493,-0.000000,-1.386084,-1.385888,0.289523,0.000000,...,1.000000,2.000000,470.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.000000,3.000000
max,4.087440e+03,4087.440000,155.161000,4.704100e+03,4704.100000,155.161000,-0.004182,-0.003921,4087.360000,875.123000,...,5.000000,12.000000,6300.000000,284.616000,24053.000000,14.391200,38.043100,0.024000,2.000000,3.000000


CPU times: user 7.62 s, sys: 1.36 s, total: 8.98 s
Wall time: 8.99 s


In [8]:
mdg_series = {
    'ER.H2O.FWTL.ZS': 'Annual freshwater withdrawals, total (% of internal resources)',
    'ER.H2O.FWTL.K3': 'Annual freshwater withdrawals, total (billion cubic meters)',
    'ER.H2O.FWST.ZS': 'Level of water stress: freshwater withdrawal as a proportion of available freshwater resources',
    'SH.H2O.BASW.ZS': 'People using at least basic drinking water services (% of population)',
    'SG.H2O.TM30.HH.ZS': 'Households with water 30 minutes or longer away round trip (%)',
    'SG.H2O.TL30.HH.ZS': 'Households with water less than 30 minutes away round trip (%)',
    'SG.H2O.PRMS.HH.ZS': 'Households with water on the premises (%)',
    # 'SH.H2O.SAFE.ZS': 'Improved water source (% of population with access)', # not accessibale -> archiv
    'SH.H2O.SMDW.ZS': 'People using safely managed drinking water services (% of population)',
    # 'SH.STA.ACSN': 'Improved sanitation facilities (% of population with access)', #  not accessibale -> archiv
    'SP.POP.TOTL': 'Population, total',
    'SH.STA.BRTC.ZS': 'Births attended by skilled health staff (% of total)',
    #'SH.STA.ACSN': 'Improved sanitation facilities (% of population with access)',
    #'SH.H2O.SAFE.ZS': 'Improved water source (% of population with access)',
    'SP.DYN.LE00.IN': 'Life expectancy at birth, total (years)',
    'SH.STA.ANVC.ZS': 'Pregnant women receiving prenatal care (%)',
    'SH.STA.ANV4.ZS': 'Pregnant women receiving prenatal care of at least four visits (% of pregnant women)',
    'EG.ELC.ACCS.ZS': 'Access to electricity (% of population)',
    'AG.LND.ARBL.HA.PC': 'Arable land (hectares per person)',
    'MS.MIL.TOTL.TF.ZS': 'Armed forces personnel (% of total labor force)',
    'SL.TLF.0714.WK.TM': 'Average working hours of children, working only, ages 7-14 (hours per week)',
    'SL.MNF.0714.ZS': 'Child employment in manufacturing (% of economically active children ages 7-14)',
    'SL.AGR.0714.ZS': 'Child employment in agriculture (% of economically active children ages 7-14)',
    'SL.SRV.0714.ZS': 'Child employment in services (% of economically active children ages 7-14)',
    'SL.TLF.0714.ZS': 'Children in employment, total (% of children ages 7-14)',
    'SE.COM.DURS': 'Compulsory education, duration (years)',
    'SE.ADT.LITR.ZS': 'Literacy rate, adult total (% of people ages 15 and above)',
    'SH.STA.HYGN.ZS': 'People with basic handwashing facilities including soap and water (% of population)',
    'SH.ALC.PCAP.LI': 'Total alcohol consumption per capita (liters of pure alcohol, projected estimates, 15+ years of age)',
    'SG.VAW.REAS.ZS': 'Women who believe a husband is justified in beating his wife (%)',
#    'IN.POV.HH.DRNKNGWATER.TOTL.PCT': 'Households Access to Safe Drinking Water - Total (%) (india)', # Country Partnership Strategy for India (FY2013 - 17)
#    'IN.POV.HH.DRKNGWATER.WITHIN': 'Availability of drinking water source within the premises (india)',
#    'IN.POV.HH.DRKNGWATER.NEAR': 'Availability of drinking water source near the premises (india)',
#    'IN.POV.HH.DRKNGWATER.AWAY': 'Availability of drinking water from a source away (india)',
#    'IN.POV.HH.DRKNGWATER': 'Total households with drinking water facility (india)',
    'IE.PPN.WATR.CD': 'Public private partnerships investment in water and sanitation (current US$)',
    'IE.PPI.WATR.CD': 'Investment in water and sanitation with private participation (current US$)',
    'IC.WAT.GIFT.ZS': 'Expected to give gifts to get a water connection (% of firms)',
    'IC.FRM.CORR.CORR7': 'Percent of firms expected to give gifts to get a water connection',
    'IC.WAT.DURS': 'Delay in obtaining a water connections (days)',
    'IC.FRM.INFRA.IN6':'Number of water insufficiencies in a typical month',
#    'HOU.H2O.ACSN.ZS': 'Household Access to Safe Water (in % of total household) (indonesia)',
    'ER.H2O.INTR.PC': 'Renewable internal freshwater resources per capita (cubic meters)',
    'ER.H2O.INTR.K3': 'Renewable internal freshwater resources, total (billion cubic meters)',
#    '5.51.01.09.water': 'Access to water' # usless data
    'SI.POV.XPND.MD': 'Median daily per capita income or consumption expenditure (2011 PPP)',
    'SI.SPR.PCAP': 'Survey mean consumption or income per capita, total population (2011 PPP $ per day)',
    'AG.SRF.TOTL.K2': 'Surface area (sq. km)'
    
}

wdf = fetch_series(series=mdg_series,date="1999:2018")
countrynames = get_all_countries(cachedir=cachedir,includeaggs=True)
wdf.rename(columns=mdg_series,inplace=True)
#display(countrynames)


Reading Data from cached file: data/cache/ER.H2O.FWTL.ZS.p
Reading Data from cached file: data/cache/ER.H2O.FWTL.K3.p
Reading Data from cached file: data/cache/ER.H2O.FWST.ZS.p
Reading Data from cached file: data/cache/SH.H2O.BASW.ZS.p
Reading Data from cached file: data/cache/SG.H2O.TM30.HH.ZS.p
Reading Data from cached file: data/cache/SG.H2O.TL30.HH.ZS.p
Reading Data from cached file: data/cache/SG.H2O.PRMS.HH.ZS.p
Reading Data from cached file: data/cache/SH.H2O.SMDW.ZS.p
Reading Data from cached file: data/cache/SP.POP.TOTL.p
Reading Data from cached file: data/cache/SH.STA.BRTC.ZS.p
Reading Data from cached file: data/cache/SP.DYN.LE00.IN.p
Reading Data from cached file: data/cache/SH.STA.ANVC.ZS.p
Reading Data from cached file: data/cache/SH.STA.ANV4.ZS.p
Reading Data from cached file: data/cache/EG.ELC.ACCS.ZS.p
Reading Data from cached file: data/cache/AG.LND.ARBL.HA.PC.p
Reading Data from cached file: data/cache/MS.MIL.TOTL.TF.ZS.p
Reading Data from cached file: data/cache/SL

In [9]:
for i in itcc:
    d = defaultdict(dict)
    tmp = wdf[wdf['Country'] == i].copy()
    tmp = tmp.replace(0.0,np.nan)
    tmp = tmp.drop(columns=['Country'])
    tmp = tmp.set_index('Year')
    for j in tmp.columns:
        tmp2 = tmp[j]
        tmp2 = tmp2[tmp2.notna()].tail(1)
        
        #display(tmp2.reset_index().tail(1))
        #display(tmp2)
        tmp2 = tmp2.reset_index()
        #display(tmp2)
        try:
            d[j]['Year']=tmp2['Year'][0]
            d[j]['Value']=tmp2[j][0]
        except:
            d[j]['Year'] = np.nan
            d[j]['Value'] = np.nan
            pass
    htmlout = """<h2>%s<h2>
    <table>
    <tr><th>Stat</th><th>Year</th><th>Value</th></tr>
    """ %(itcc[i])
    for j in tmp.columns:
        htmlout = htmlout + """
        <tr><td> %s </td><td> %s </td><td> %.2f </td></tr>
        """ %(j.replace("$","&#36;").replace("%","&#37;"),d[j]['Year'],d[j]['Value'])
    
    ### some stats on grants
    for k in ['USD_Commitment_Defl','USD_Disbursement_Defl']:
        tmp = df[df['RecipientCode'] == codemapping[i]].copy()
        tmp = tmp[tmp["CommitmentDate"] > datetime(year=2001,month=12,day=31)]
        tmp = tmp[tmp["CommitmentDate"] < datetime(year=2020,month=1,day=1)]
        j = 'sum of %s mUSD' %k
        d[j]['Year'] = "2002-2019"
        d[j]['Value'] = float(tmp[k].sum())
        htmlout = htmlout + """
        <tr><td> %s </td><td> %s </td><td> %.2f </td></tr>
        """ %(j,d[j]['Year'],d[j]['Value'])

        j = 'sum of %s mUSD for water' %k
        d[j]['Year'] = "2002-2019"
        d[j]['Value'] = float(tmp[tmp['SectorCode']=='140'][k].sum())
        htmlout = htmlout + """
        <tr><td> %s </td><td> %s </td><td> %.2f </td></tr>
        """ %(j,d[j]['Year'],d[j]['Value'])

        tmp = df[df['RecipientCode'] == codemapping[i]].copy()
        tmp = tmp[tmp['DonorCode'] == "5"]
        tmp = tmp[tmp["CommitmentDate"] > datetime(year=2001,month=12,day=31)]
        tmp = tmp[tmp["CommitmentDate"] < datetime(year=2020,month=1,day=1)]
        j = 'sum of %s mUSD by germany' %k
        d[j]['Year'] = "2002-2019"
        d[j]['Value'] = float(tmp[k].sum())
        htmlout = htmlout + """
        <tr><td> %s </td><td> %s </td><td> %.2f </td></tr>
        """ %(j,d[j]['Year'],d[j]['Value'])

        j = 'sum of %s mUSD for water germany' %k
        d[j]['Year'] = "2002-2019"
        d[j]['Value'] = float(tmp[tmp['SectorCode']=='140'][k].sum())
        htmlout = htmlout + """
        <tr><td> %s </td><td> %s </td><td> %.2f </td></tr>
        """ %(j,d[j]['Year'],d[j]['Value'])

        ### second time frame

        tmp = df[df['RecipientCode'] == codemapping[i]].copy()
        tmp = tmp[tmp["CommitmentDate"] > datetime(year=2011,month=12,day=31)]
        tmp = tmp[tmp["CommitmentDate"] < datetime(year=2020,month=1,day=1)]
        j = 'sum of %s mUSD' %k
        d[j]['Year'] = "2012-2019"
        d[j]['Value'] = float(tmp[k].sum())
        htmlout = htmlout + """
        <tr><td> %s </td><td> %s </td><td> %.2f </td></tr>
        """ %(j,d[j]['Year'],d[j]['Value'])

        j = 'sum of %s mUSD for water' %k
        d[j]['Year'] = "2012-2019"
        d[j]['Value'] = float(tmp[tmp['SectorCode']=='140'][k].sum())
        htmlout = htmlout + """
        <tr><td> %s </td><td> %s </td><td> %.2f </td></tr>
        """ %(j,d[j]['Year'],d[j]['Value'])

        tmp = df[df['RecipientCode'] == codemapping[i]].copy()
        tmp = tmp[tmp['DonorCode'] == "5"]
        tmp = tmp[tmp["CommitmentDate"] > datetime(year=2011,month=12,day=31)]
        tmp = tmp[tmp["CommitmentDate"] < datetime(year=2020,month=1,day=1)]
        j = 'sum of %s mUSD by germany' %k
        d[j]['Year'] = "2012-2019"
        d[j]['Value'] = float(tmp[k].sum())
        htmlout = htmlout + """
        <tr><td> %s </td><td> %s </td><td> %.2f </td></tr>
        """ %(j,d[j]['Year'],d[j]['Value'])

        j = 'sum of %s mUSD for water by germany' %k
        d[j]['Year'] = "2012-2019"
        d[j]['Value'] = float(tmp[tmp['SectorCode']=='140'][k].sum())
        htmlout = htmlout + """
        <tr><td> %s </td><td> %s </td><td> %.2f </td></tr>
        """ %(j,d[j]['Year'],d[j]['Value'])
        
    htmlout = htmlout + "</table>"
    display(HTML(htmlout))
    

Stat,Year,Value
"Annual freshwater withdrawals, total (% of internal resources)",2014,124.49
"Annual freshwater withdrawals, total (billion cubic meters)",2015,1.10
Level of water stress: freshwater withdrawal as a proportion of available freshwater resources,2015,150.90
People using at least basic drinking water services (% of population),2017,98.94
Households with water 30 minutes or longer away round trip (%),2018,0.10
Households with water less than 30 minutes away round trip (%),2018,0.40
Households with water on the premises (%),2018,99.30
People using safely managed drinking water services (% of population),2017,93.82
"Population, total",2019,10101694.00
Births attended by skilled health staff (% of total),2018,99.70


Stat,Year,Value
"Annual freshwater withdrawals, total (% of internal resources)",2014,4100.00
"Annual freshwater withdrawals, total (billion cubic meters)",2010,73.80
Level of water stress: freshwater withdrawal as a proportion of available freshwater resources,2014,159.85
People using at least basic drinking water services (% of population),2017,99.11
Households with water 30 minutes or longer away round trip (%),2014,0.60
Households with water less than 30 minutes away round trip (%),2014,6.10
Households with water on the premises (%),2014,93.20
People using safely managed drinking water services (% of population),nan,nan
"Population, total",2019,100388073.00
Births attended by skilled health staff (% of total),2014,91.50


Stat,Year,Value
"Annual freshwater withdrawals, total (% of internal resources)",2012,1.63
"Annual freshwater withdrawals, total (billion cubic meters)",2008,0.64
Level of water stress: freshwater withdrawal as a proportion of available freshwater resources,2014,1.33
People using at least basic drinking water services (% of population),2017,49.10
Households with water 30 minutes or longer away round trip (%),2019,29.50
Households with water less than 30 minutes away round trip (%),2019,49.30
Households with water on the premises (%),2019,20.20
People using safely managed drinking water services (% of population),2017,7.07
"Population, total",2019,44269594.00
Births attended by skilled health staff (% of total),2016,74.20


Stat,Year,Value
"Annual freshwater withdrawals, total (% of internal resources)",2002,1.58
"Annual freshwater withdrawals, total (billion cubic meters)",2000,0.15
Level of water stress: freshwater withdrawal as a proportion of available freshwater resources,2014,1.44
People using at least basic drinking water services (% of population),2017,57.71
Households with water 30 minutes or longer away round trip (%),2017,27.00
Households with water less than 30 minutes away round trip (%),2017,58.40
Households with water on the premises (%),2017,14.10
People using safely managed drinking water services (% of population),nan,nan
"Population, total",2019,12626950.00
Births attended by skilled health staff (% of total),2015,90.70


Stat,Year,Value
"Annual freshwater withdrawals, total (% of internal resources)",2014,15.55
"Annual freshwater withdrawals, total (billion cubic meters)",2010,3.22
Level of water stress: freshwater withdrawal as a proportion of available freshwater resources,2014,14.31
People using at least basic drinking water services (% of population),2017,58.92
Households with water 30 minutes or longer away round trip (%),2015,12.10
Households with water less than 30 minutes away round trip (%),2015,39.50
Households with water on the premises (%),2015,47.50
People using safely managed drinking water services (% of population),nan,nan
"Population, total",2019,52573973.00
Births attended by skilled health staff (% of total),2014,61.80


Stat,Year,Value
"Annual freshwater withdrawals, total (% of internal resources)",2014,673.25
"Annual freshwater withdrawals, total (billion cubic meters)",2011,26.93
Level of water stress: freshwater withdrawal as a proportion of available freshwater resources,2014,93.67
People using at least basic drinking water services (% of population),2017,60.27
Households with water 30 minutes or longer away round trip (%),nan,nan
Households with water less than 30 minutes away round trip (%),nan,nan
Households with water on the premises (%),nan,nan
People using safely managed drinking water services (% of population),nan,nan
"Population, total",2019,42813238.00
Births attended by skilled health staff (% of total),2014,77.50


Stat,Year,Value
"Annual freshwater withdrawals, total (% of internal resources)",2014,44.78
"Annual freshwater withdrawals, total (billion cubic meters)",2010,647.50
Level of water stress: freshwater withdrawal as a proportion of available freshwater resources,2014,44.53
People using at least basic drinking water services (% of population),2017,92.67
Households with water 30 minutes or longer away round trip (%),2016,2.30
Households with water less than 30 minutes away round trip (%),2016,31.40
Households with water on the premises (%),2016,66.30
People using safely managed drinking water services (% of population),nan,nan
"Population, total",2019,1366417754.00
Births attended by skilled health staff (% of total),2016,81.40


Stat,Year,Value
"Annual freshwater withdrawals, total (% of internal resources)",2012,34.16
"Annual freshwater withdrawals, total (billion cubic meters)",2008,35.87
Level of water stress: freshwater withdrawal as a proportion of available freshwater resources,2014,3.79
People using at least basic drinking water services (% of population),2017,97.02
Households with water 30 minutes or longer away round trip (%),2014,1.60
Households with water less than 30 minutes away round trip (%),2014,22.70
Households with water on the premises (%),2014,75.50
People using safely managed drinking water services (% of population),2017,55.44
"Population, total",2019,163046161.00
Births attended by skilled health staff (% of total),2016,49.80


Stat,Year,Value
"Annual freshwater withdrawals, total (% of internal resources)",2012,0.55
"Annual freshwater withdrawals, total (billion cubic meters)",2008,11.77
Level of water stress: freshwater withdrawal as a proportion of available freshwater resources,2014,0.86
People using at least basic drinking water services (% of population),2017,97.30
Households with water 30 minutes or longer away round trip (%),2015,0.60
Households with water less than 30 minutes away round trip (%),2015,3.30
Households with water on the premises (%),2015,95.90
People using safely managed drinking water services (% of population),2017,73.23
"Population, total",2019,50339443.00
Births attended by skilled health staff (% of total),2016,95.50


Stat,Year,Value
"Annual freshwater withdrawals, total (% of internal resources)",2012,0.69
"Annual freshwater withdrawals, total (billion cubic meters)",2009,2.09
Level of water stress: freshwater withdrawal as a proportion of available freshwater resources,2014,0.53
People using at least basic drinking water services (% of population),2017,92.85
Households with water 30 minutes or longer away round trip (%),2008,0.70
Households with water less than 30 minutes away round trip (%),2008,11.20
Households with water on the premises (%),2008,88.00
People using safely managed drinking water services (% of population),nan,nan
"Population, total",2019,11513100.00
Births attended by skilled health staff (% of total),2016,89.80
